In [2]:
import sys
import os
from pathlib import Path

project_root = Path(__file__).parent.parent if '__file__' in globals() else Path.cwd().parent
sys.path.insert(0, str(project_root))

from core.backtesting_opt import _Data, Strategy, Backtest
from utils.functions import BlackScholes as bs
import pandas as pd
import numpy as np
import math
from scipy.optimize import root_scalar
from datetime import datetime, date as DateObject



In [3]:
class IV_Slope(Strategy):
    upper_threshold = 0.19
    lower_threshold = 0.15
    position_id= 0
    
    def init(self):
        self.legs = {
            'leg1': {'type': 'CE', 'expiry_type': 'weekly', 'expiry_range': [12, 20], 'target_strike': 'ATM', 'stop_loss': None, 'take_profit': None},
            'leg2': {'type': 'PE', 'expiry_type': 'weekly', 'expiry_range': [12, 20], 'target_strike': 'ATM', 'stop_loss': None, 'take_profit': None},
            'leg3': {'type': 'CE', 'expiry_type': 'monthly', 'expiry_range': [26, 34], 'target_strike': 'ATM', 'stop_loss': None, 'take_profit': None},
            'leg4': {'type': 'PE', 'expiry_type': 'monthly', 'expiry_range': [26, 34], 'target_strike': 'ATM', 'stop_loss': None, 'take_profit': None}
        }
        # Flag to track if we've already executed today
        self.executed_today = False
        self.last_execution_date = None
        self.previous_signal = 0
    
    def execution_window(self, start_time_str="14:30:00", end_time_str="15:00:00"):
        current_time = pd.Timestamp(self.time).time()
        current_date = pd.Timestamp(self.time).date()
        
        # Reset execution flag for new trading day
        if self.last_execution_date != current_date:
            self.executed_today = False
            self.last_execution_date = current_date
        
        # Check if current time is within execution window
        start_time = pd.Timestamp(start_time_str).time()
        end_time = pd.Timestamp(end_time_str).time()
        
        if not (start_time <= current_time <= end_time):
            return False
            
        # Execute only once per day during the time window
        if self.executed_today:
            return False
            
        # Mark as executed for today
        self.executed_today = True
        return True
    
    def next(self):
        if self.spot is None or pd.isna(self.spot):
            return
            
        # Check if we should execute today
        if not self.execution_window():
            return
        
        atm = round(self.spot / 50) * 50
        
        for leg in self.legs.values():
            available_ttes = list(self.tte_to_expiry.keys()) if self.tte_to_expiry else []
            lower, upper = leg["expiry_range"]
            matching_ttes = [tte for tte in available_ttes if lower <= tte <= upper]
            if not matching_ttes:
                print(f"No matching TTEs found for leg {leg['type']} with range {leg['expiry_range']}. Available TTEs: {sorted(available_ttes) if available_ttes else 'None'}")
                return 
            
            valid_tte = min(matching_ttes)
            leg["expiry"] = self.tte_to_expiry[valid_tte]
            
            leg["strike"] = float(atm)
            contract = f"NIFTY{pd.Timestamp(leg['expiry']).strftime('%d%b%y').upper()}{int(leg['strike'])}{leg['type']}"
            leg["contract"] = contract
            leg["data"] = self._data.get_ticker_data(contract)
        print(self.legs)
        missing_legs = [leg["contract"] for leg in self.legs.values() if leg["data"] is None]
        if missing_legs:
            print(f"IV not found for {self.time}. Spot: {self.spot} Missing legs: {missing_legs}")
            return
        
        iv_slope = math.log((self.legs["leg1"]["data"]["iv"] + self.legs["leg2"]["data"]["iv"]) / (self.legs["leg3"]["data"]["iv"] + self.legs["leg4"]["data"]["iv"]), 10)
        
        active_trades = self.active_trades
        new_signal = 0
        if iv_slope > self.upper_threshold:
            new_signal = -1
        elif iv_slope < self.lower_threshold:
            new_signal = 1
        
        if new_signal == -1:
            self.entry_type_dict = {'weekly': 'SELL', 'monthly': 'BUY'}
        elif new_signal == 1:
            self.entry_type_dict = {'weekly': 'BUY', 'monthly': 'SELL'}
        elif new_signal == 0:
            return
            
        
        if not active_trades:
            
            placed_any_leg = False
            for leg_id, leg in self.legs.items():
                entry_type = self.entry_type_dict.get(leg["expiry_type"])
                order_fn = {'BUY': self.buy, 'SELL': self.sell}.get(entry_type)
                if order_fn is None:
                    continue
                order_fn(
                    strategy_id='strat1',
                    position_id=self.position_id,
                    leg_id=leg_id,
                    ticker=leg["contract"],
                    quantity=1,
                    stop_loss=None,
                    take_profit=None,
                    tag=f'{new_signal} signal entry'
                )
                placed_any_leg = True
            if placed_any_leg:
                self.position_id += 1
                self.previous_signal = new_signal
        else:
            near_expiry = None
            for trade in active_trades:
                expiry = datetime.strptime(trade.ticker[-14:-7], "%d%b%y").date()
                near_expiry = expiry if near_expiry is None else min(near_expiry, expiry)
            
            exit_reason = (
                "Near Expiry reached" if (pd.Timestamp(self.time).date() == near_expiry) else
                "Signal changed" if (self.previous_signal != new_signal) else
                None
            )
            
            if exit_reason:
                for trade in active_trades:
                    print("Closing position")
                    trade.close(trade.size, tag=exit_reason)
                self.previous_signal = 0
                
                placed_any_leg = False
                for leg_id, leg in self.legs.items():
                    entry_type = self.entry_type_dict.get(leg["expiry_type"])
                    order_fn = {'BUY': self.buy, 'SELL': self.sell}.get(entry_type)
                    if order_fn is None:
                        continue
                    order_fn(
                        strategy_id='strat1',
                        position_id=self.position_id,
                        leg_id=leg_id,
                        ticker=leg["contract"],
                        quantity=1,
                        stop_loss=None,
                        take_profit=None,
                        tag=f'{new_signal} signal entry'
                    )
                    placed_any_leg = True
                if placed_any_leg:
                    self.position_id += 1
                    self.previous_signal = new_signal
                
                
            else:
                if self.previous_signal == new_signal:
                    leg_strike = self.legs["leg4"]["strike"]
                    if (0.99*leg_strike) <= self.spot <= (1.01*leg_strike):
                        pass
                    else:
                            
                            # Place orders according to entry type
                            placed_any_leg = False
                            for leg_id, leg in self.legs.items():
                                entry_type = self.entry_type_dict.get(leg["expiry_type"])
                                order_fn = {'BUY': self.buy, 'SELL': self.sell}.get(entry_type)
                                if order_fn is None:
                                    continue
                                print(f"Placing {entry_type} order for {leg_id}: {leg['contract']}")
                                order_fn(
                                    strategy_id='strat1',
                                    position_id=self.position_id,
                                    leg_id=leg_id,
                                    ticker=leg["contract"],
                                    quantity=1,
                                    stop_loss=None,
                                    take_profit=None,
                                    tag=f'Adjustment {entry_type} at ATM {atm}'
                                )
                                placed_any_leg = True
                            
                            if placed_any_leg:
                                self.position_id += 1

In [4]:
# db_path = "nifty_1min_desiquant.duckdb"
db_path = "nifty_opt_icici_1min.duckdb"

bt = Backtest(db_path=db_path, strategy=IV_Slope, cash=10000000, commission_per_contract=0.65, option_multiplier=75)
stats = bt.run(start_date="2022-01-01", end_date="2024-01-01")
print(stats)

Identifying tables in date range...
Found 468 tables out of 966 total tables with data in range


Backtesting Options Strategy:   0%|          | 2/468 [00:00<00:23, 19.53it/s]

{'leg1': {'type': 'CE', 'expiry_type': 'weekly', 'expiry_range': [12, 20], 'target_strike': 'ATM', 'stop_loss': None, 'take_profit': None, 'expiry': Timestamp('2022-01-20 21:00:00'), 'strike': 17450.0, 'contract': 'NIFTY20JAN2217450CE', 'data': None}, 'leg2': {'type': 'PE', 'expiry_type': 'weekly', 'expiry_range': [12, 20], 'target_strike': 'ATM', 'stop_loss': None, 'take_profit': None, 'expiry': Timestamp('2022-01-20 21:00:00'), 'strike': 17450.0, 'contract': 'NIFTY20JAN2217450PE', 'data': ticker             NIFTY20JAN2217450PE
open                             270.0
high                             270.0
low                              270.0
close                            270.0
volume                          1300.0
open_interest                    250.0
strike_price                   17450.0
instrument_type                     PE
expiry_date        2022-01-20 21:00:00
Time_to_expiry                    19.0
spot_price                17357.949219
r                               0.06

Backtesting Options Strategy:   1%|▏         | 6/468 [00:00<00:36, 12.72it/s]

{'leg1': {'type': 'CE', 'expiry_type': 'weekly', 'expiry_range': [12, 20], 'target_strike': 'ATM', 'stop_loss': None, 'take_profit': None, 'expiry': Timestamp('2022-01-20 21:00:00'), 'strike': 17850.0, 'contract': 'NIFTY20JAN2217850CE', 'data': ticker             NIFTY20JAN2217850CE
open                        217.300003
high                        202.949997
low                         198.800003
close                       214.050003
volume                          7400.0
open_interest                   4000.0
strike_price                   17850.0
instrument_type                     CE
expiry_date        2022-01-20 21:00:00
Time_to_expiry                    12.0
spot_price                17830.800781
r                               0.0654
iv                              0.1585
delta                           0.4911
gamma                         0.000778
vega                           12.8949
theta                          -8.5322
rho                             2.8147
Name: 2022-01-

...

{'leg1': {'type': 'CE', 'expiry_type': 'weekly', 'expiry_range': [12, 20], 'target_strike': 'ATM', 'stop_loss': None, 'take_profit': None, 'expiry': Timestamp('2023-11-16 21:00:00'), 'strike': 19250.0, 'contract': 'NIFTY16NOV2319250CE', 'data': ticker             NIFTY16NOV2319250CE
open                            182.25
high                             167.5
low                              167.5
close                       170.199997
volume                         67450.0
open_interest                  14300.0
strike_price                   19250.0
instrument_type                     CE
expiry_date        2023-11-16 21:00:00
Time_to_expiry                    12.0
spot_price                    19242.25
r                               0.0732
iv                               0.108
delta                           0.4962
gamma                         0.001059
vega                           13.9184
theta                          -6.2813
rho                             3.0908
Name: 2023-11-

/Users/vivek/Desktop/works/BACKTEST_STRATEGIES/FnO-Synapse/core/backtesting_opt.py:743: UserWarning: Cannot determine expiry for NIFTY03FEB2217800CE. Skipping expiration check.
  warnings.warn(f"Cannot determine expiry for {ticker}. Skipping expiration check.")
/Users/vivek/Desktop/works/BACKTEST_STRATEGIES/FnO-Synapse/core/backtesting_opt.py:743: UserWarning: Cannot determine expiry for NIFTY03FEB2217800PE. Skipping expiration check.
  warnings.warn(f"Cannot determine expiry for {ticker}. Skipping expiration check.")
/Users/vivek/Desktop/works/BACKTEST_STRATEGIES/FnO-Synapse/core/backtesting_opt.py:743: UserWarning: Cannot determine expiry for NIFTY03FEB2217800CE. Skipping expiration check.
  warnings.warn(f"Cannot determine expiry for {ticker}. Skipping expiration check.")
/Users/vivek/Desktop/works/BACKTEST_STRATEGIES/FnO-Synapse/core/backtesting_opt.py:743: UserWarning: Cannot determine expiry for NIFTY03FEB2217800PE. Skipping expiration check.
  warnings.warn(f"Cannot determine ex

No matching TTEs found for leg CE with range [26, 34]. Available TTEs: [2.0, 9.0, 16.0, 23.0, 44.0]
2023-11-13
No matching TTEs found for leg CE with range [26, 34]. Available TTEs: [1.0, 7.0, 14.0, 21.0, 42.0]
2023-11-15
No matching TTEs found for leg CE with range [26, 34]. Available TTEs: [1.0, 6.0, 13.0, 20.0, 41.0]
2023-11-16
{'leg1': {'type': 'CE', 'expiry_type': 'weekly', 'expiry_range': [12, 20], 'target_strike': 'ATM', 'stop_loss': None, 'take_profit': None, 'expiry': Timestamp('2023-11-30 21:00:00'), 'strike': 19750.0, 'contract': 'NIFTY30NOV2319750CE', 'data': ticker             NIFTY30NOV2319750CE
open                        157.399994
high                        137.699997
low                         137.699997
close                       157.399994
volume                        328500.0
open_interest                  20100.0
strike_price                   19750.0
instrument_type                     CE
expiry_date        2023-11-30 21:00:00
Time_to_expiry                  

/Users/vivek/Desktop/works/BACKTEST_STRATEGIES/FnO-Synapse/core/backtesting_opt.py:743: UserWarning: Cannot determine expiry for NIFTY03FEB2217800CE. Skipping expiration check.
  warnings.warn(f"Cannot determine expiry for {ticker}. Skipping expiration check.")
/Users/vivek/Desktop/works/BACKTEST_STRATEGIES/FnO-Synapse/core/backtesting_opt.py:743: UserWarning: Cannot determine expiry for NIFTY03FEB2217800PE. Skipping expiration check.
  warnings.warn(f"Cannot determine expiry for {ticker}. Skipping expiration check.")
/Users/vivek/Desktop/works/BACKTEST_STRATEGIES/FnO-Synapse/core/backtesting_opt.py:743: UserWarning: Cannot determine expiry for NIFTY03FEB2217800CE. Skipping expiration check.
  warnings.warn(f"Cannot determine expiry for {ticker}. Skipping expiration check.")
/Users/vivek/Desktop/works/BACKTEST_STRATEGIES/FnO-Synapse/core/backtesting_opt.py:743: UserWarning: Cannot determine expiry for NIFTY03FEB2217800PE. Skipping expiration check.
  warnings.warn(f"Cannot determine ex

2023-11-22
Finalizing trade for NIFTY03FEB2217800CE at price 381.3500061035156
Finalizing trade for NIFTY03FEB2217800PE at price 121.0
Start                     2022-01-03 00:00:00
End                       2023-11-22 00:00:00
Duration                    688 days 00:00:00
Exposure Time [%]                         0.0
Equity Final [$]                   10008317.0
Equity Peak [$]                    10036224.0
Return [%]                            0.08317
Return (Ann.) [%]                    0.044775
Volatility (Ann.) [%]                0.353185
Sharpe Ratio                         0.126776
Sortino Ratio                        0.188602
Calmar Ratio                         0.161026
Max. Drawdown [%]                   -0.278063
Avg. Drawdown [%]                   -0.176852
Max. Drawdown Duration      658 days 00:00:00
Avg. Drawdown Duration      226 days 00:00:00
# Trades                                    4
Win Rate [%]                              0.0
Best Trade [%]                       

In [5]:
from core.wfo import WalkForwardOptimizer

constraint = lambda x: x['upper_threshold'] > x['lower_threshold']


wfo = WalkForwardOptimizer(
    strategy=IV_Slope,
    optimization_params={
        # 'upper_threshold': [-0.15 ,-0.1, 0.1, 0.15, 0.2, 0.25, 0.3],
        # 'lower_threshold': [-0.15 ,-0.1 ,0, 0.05, 0.1, 0.15, 0.2]
        'upper_threshold': [0.1, 0.15, 0.2, 0.25],
        'lower_threshold': [ 0.1, 0.15, 0.2]
    },
    # constraint=constraint,
    maximize='Sharpe Ratio',
)

Successfully imported local quantstats from: /Users/vivek/Desktop/works/BACKTEST_STRATEGIES/FnO-Synapse/core/quantstats
Successfully imported plotting module from local quantstats


In [8]:
# def optimize_stock(self, stock, timeframe, db_path, exchange = None, training_candles = 2000, testing_candles = 200):
    
wfo.optimize_stock(
    stock='NIFTY',
    timeframe='1min',
    db_path='../core/nifty_opt_icici_1min.duckdb',
    exchange='NSE',
    training_candles=6000,
    testing_candles=3000,
    start_date='2022-01-01',
    end_date='2024-01-01',
    # method='sambo',
    # max_tries=10
)

Using date range: 2022-01-01 to 2024-01-01


Walk-Forward Optimizing NIFTY:   0%|          | 0/89 [00:00<?, ?it/s]

Identifying tables in date range...
Found 0 tables out of 0 total tables with data in range


In [9]:
bt.tear_sheet()

Created default benchmark with 467 data points
Filtered trade_stats_df:                                      Value
Statistic                                 
Start                  2022-01-03 00:00:00
End                    2023-11-22 00:00:00
Duration                 688 days 00:00:00
Exposure Time [%]                      0.0
Equity Final [$]                10008317.0
Equity Peak [$]                 10036224.0
Return [%]                         0.08317
Return (Ann.) [%]                 0.044775
Volatility (Ann.) [%]             0.353185
Sharpe Ratio                      0.126776
Sortino Ratio                     0.188602
Calmar Ratio                      0.161026
Max. Drawdown [%]                -0.278063
# Trades                                 4
Win Rate [%]                           0.0
Profit Factor                          0.0
Expectancy [%]                  -44.802521
Kelly Criterion                        NaN
No backtest object provided. Skipping equity curve generation.
HTML 